In [2]:
from util.conversion_util import Conversion
from util.db_util import Database
import pandas as pd

# girls
xstandards = {
    "4 x 800 Relay": "9:23.63",
    "100 Hurdles": "14.81",          
    "100 Meters": "12.24",
    "1600 Meters": "4:58.87",
    "4 x 100 Relay": "48.41",
    "400 Meters": "57.44",
    "300 Hurdles": "45.44",
    "800 Meters": "2:14.68",
    "200 Meters": "25.34",
    "3200 Meters": "10:47.57",
    "4 x 400 Relay": "3:58.35",
    "Discus": "127' 3\"",
    "Shot Put": "42' 9.25\"",
    "Long Jump": "18' 0.25\"",
    "High Jump": "5' 4\"",
    "Pole Vault": "11' 3\""
}


# boys
standards = {
    "4 x 800 Relay": "7:56.28",
    "110 Hurdles": "14.77",
    "100 Meters": "11.01",
    "1600 Meters": "4:16.70",
    "4 x 100 Relay": "42.44",
    "400 Meters": "49.07",
    "300 Hurdles": "39.35",
    "800 Meters": "1:55.21",
    "200 Meters": "22.21",
    "3200 Meters": "9:10.91",
    "4 x 400 Relay": "3:21.92",  
    "Discus": "161' 4\"",
    "Shot Put": "56' 4\"",
    "Long Jump": "21' 11.5\"",
    "High Jump": "6' 5\"",
    "Pole Vault": "14' 6\""
}


c = Conversion()

for key, value in standards.items():
    # if track event
    if(key[0].isdigit()):
        standards[key] = c.time_to_seconds(value)
    else:
        standards[key] = c.distance_to_inches(value)

print(standards)
    

{'4 x 800 Relay': 476.28, '110 Hurdles': 14.77, '100 Meters': 11.01, '1600 Meters': 256.7, '4 x 100 Relay': 42.44, '400 Meters': 49.07, '300 Hurdles': 39.35, '800 Meters': 115.21, '200 Meters': 22.21, '3200 Meters': 550.91, '4 x 400 Relay': 201.92, 'Discus': 1936.0, 'Shot Put': 676.0, 'Long Jump': 263.5, 'High Jump': 77.0, 'Pole Vault': 174.0}


In [23]:
db = Database("db/Track.db")
c = Conversion()

for key in standards:
    res = db.get_state_standard(2025, "Boys", key)

    if key[0].isdigit():     
        print(f"{key:<15}  {c.seconds_to_time(res)}")
    else:
        print(f"{key:<15}  {c.inches_to_distance(res)}")


4 x 800 Relay    7:56.28
110 Hurdles      14.77
100 Meters       10.90
1600 Meters      4:16.70
4 x 100 Relay    42.44
400 Meters       49.07
300 Hurdles      39.35
800 Meters       1:55.21
200 Meters       22.09
3200 Meters      9:10.91
4 x 400 Relay    3:21.92
Discus           161' 4"
Shot Put         56' 4.25"
Long Jump        21' 11.75"
High Jump        6' 5"
Pole Vault       14' 6"


In [25]:

YEAR = 2025
QUALIFIERS_TO_PROCESS = "State" # Regional or State
GENDER_LIST = ['Boys'] # Girls and/or Boys

# are we processing State or Regional qualifiers
if QUALIFIERS_TO_PROCESS == "State":
    MEET_TYPE = 'Regional'
    CALLBACKS = 3
else:
    MEET_TYPE = 'Sectional'
    CALLBACKS = 4


In [27]:
# get individual results
athlete_df = db.get_all_athlete_results()
athlete_df = athlete_df[(athlete_df['meet_type'] == MEET_TYPE) & (athlete_df['result_type'] == "Final") & (athlete_df['year'] == YEAR)].copy()
athlete_df['place'] = athlete_df['place'].astype('Int64')
athlete_df.loc[:, 'name'] = athlete_df['last'] + ", " + athlete_df['first']
athlete_df = athlete_df.rename(columns={'name': 'Name', 'grade': 'Grade', "school_name": "School", "result": "Result"})

# get relay results
relay_df = db.get_all_relay_results()
relay_df = relay_df[(relay_df['meet_type'] == MEET_TYPE) & (relay_df['year'] == YEAR)]
relay_df['place'] = relay_df['place'].astype('Int64')
relay_df = relay_df.rename(columns={"school_name": "School", "result": "Result"})

In [29]:

if MEET_TYPE == "Sectional":
    meets = [(1,2,3,4),(5,6,7,8),(9,10,11,12),(13,14,15,16),(17,18,19,20),(21,22,23,24),(25,26,27,28),(29,30,31,32)]
else:
    meets = [(1,2,3,4,5,6,7,8)]

# determine max field lengths to for printing formatting 
PADDING = 5
SCHOOL_LEN = max(len(relay_df.loc[relay_df['School'].str.len().idxmax(), 'School']), len(athlete_df.loc[athlete_df['School'].str.len().idxmax(), 'School'])) + PADDING
ATHLETE_LEN = len(athlete_df.loc[athlete_df['Name'].str.len().idxmax(), 'Name']) + PADDING
RESULT_LEN = max(len(athlete_df.loc[athlete_df['Result'].str.len().idxmax(), 'Result']), len(relay_df.loc[relay_df['Result'].str.len().idxmax(), 'Result'])) + PADDING
GRADE_LEN = 2 + PADDING
PLACE_LEN = 1 + PADDING

# iterate through all the meets
for m in meets:

    # iterate through all the genders
    for g in GENDER_LIST:

        meet_num = m[3]/4

        if MEET_TYPE == "Sectional":
            filename = g + " Regional #" + str(int(meet_num)) + ".txt"
        else:
            filename = g + " State.txt"
        
        with open(filename, "w") as file:
            file.write(filename[:-4] + "\n") 
            file.write("This is an unofficial list of qualifiers. The list was generated by taking the top three finishers from each " + MEET_TYPE + ", followed by the callbacks. An asterisk (*) next to an athlete's name indicates a callback. This list does not account for possible scratches that may occur after " + MEET_TYPE + "s.\n")
        
        # specify events and order to process events
        events = ["100 Meters", "200 Meters", "400 Meters", "800 Meters", "1600 Meters", "3200 Meters", "100 Hurdles", "300 Hurdles", "High Jump", "Long Jump", "Shot Put", "Discus", "Pole Vault", "4 x 100 Relay", "4 x 400 Relay", "4 x 800 Relay"]      
   
        if g == "Boys":
            events[6] = "110 Hurdles"
        else:
            events[6] = "100 Hurdles"

        master_df = pd.DataFrame()
        
        # iterate through all the events
        for e in events:
 
            # set sort order and event type (Relay, Track, or Field)
            if(e[0].isdigit()):
                sort_ascending = True
                
                if "Relay" in e:
                    event_type = "Relay"
                else:
                    event_type = "Track"
            else:
                sort_ascending = False
                event_type = "Field"
            
            # set dataframes and columns based off individual or relay processing
            if event_type == "Relay":
                podium_df = relay_df[relay_df["place"] <= 3]
                other_df = relay_df[(relay_df["place"] > 3)]
                col_widths = {'School': SCHOOL_LEN, 'Result' : RESULT_LEN, 'place': PLACE_LEN, 'host': SCHOOL_LEN}
                col_list = ['School', 'Result', 'place', 'host']
            else:
                podium_df = athlete_df[athlete_df["place"] <= 3]
                other_df = athlete_df[(athlete_df["place"] > 3)]
                col_widths = {'Name': ATHLETE_LEN, 'Grade': GRADE_LEN, 'School': SCHOOL_LEN, 'Result' : RESULT_LEN, 'place': PLACE_LEN, 'host' : SCHOOL_LEN }
                col_list = ['Name', 'Grade', 'School', 'Result', 'place', 'host']
            
            # get top 3 place finishers and potential callbacks
            top3_df = podium_df.query("gender == @g and event == @e and meet_num in @m")

            # event standard
            standard = standards[e]
            
            # get any performances that were not in top 3 but were a state standard
            if event_type == "Field":
                state_standard_df = other_df.query("gender == @g and event == @e and meet_num in @m and result2 >= @standard")
                remaining_df = other_df.query("gender == @g and event == @e and meet_num in @m and result2 < @standard")
            else:
                state_standard_df = other_df.query("gender == @g and event == @e and meet_num in @m and result2 <= @standard")
                remaining_df = other_df.query("gender == @g and event == @e and meet_num in @m and result2 > @standard")

            # mark the rows as advancing by STAN
            state_standard_df.loc[:,'place'] = "#"
            
            # determine number of callback spots available 
            num_callbacks = max(CALLBACKS - len(state_standard_df), 0)

            if num_callbacks > 0:
                callback_df = remaining_df.query("gender == @g and event == @e and meet_num in @m")
                
                callback_df.loc[:,'place'] = "*" 
                
                # sort callbacks
                sorted_callback_df = callback_df.sort_values(by='result2', ascending=sort_ascending)
            
                # process possible ties with callback
                last_callback_result = sorted_callback_df.iloc[num_callbacks-1]['result2']

                done = False
                while(not done):
                    current_result = sorted_callback_df.iloc[num_callbacks]['result2']

                    # does the current result tie the last callback result?
                    if current_result == last_callback_result: 
                        num_callbacks = num_callbacks + 1
                    else:
                        done = True                            

            # put all qualifiers into a dataframe
            if num_callbacks > 0:
                qualifiers_df = pd.concat([top3_df, state_standard_df, sorted_callback_df.head(num_callbacks).copy()], ignore_index=True)
            else:
                qualifiers_df = pd.concat([top3_df, state_standard_df], ignore_index=True)
  
            # sort qualifiers
            sorted_qualifiers_df = qualifiers_df.sort_values(by='result2', ascending=sort_ascending)
            sorted_qualifiers_df['host'] = sorted_qualifiers_df['host'].astype(str) + " (" + sorted_qualifiers_df['place'].astype(str) + ")"
            
            # add this event qualifiers to the master dataframe of all the qualifiers
            master_df = pd.concat([master_df, sorted_qualifiers_df], ignore_index=True)

            # append qualifiers to the file
            with open(filename, "a") as file:
                file.write("\n")
                file.write("*** Event " + e + " ***\n")
                    
                # format and print each row
                header = "".join(f"{col:<{col_widths[col]}}" for col in col_list)
                file.write(header + "\n")
                file.write("-" * len(header) + "\n")
                    
                for _, row in sorted_qualifiers_df.iterrows():
                    line = "".join(f"{str(row[col]):<{col_widths[col]}}" for col in col_list)
                    file.write(line + "\n")
        

In [31]:
master_df

place_values = {
    1: 10,
    2: 8,
    3: 7,
    4: 6,
    5: 5,
    6: 4,
    7: 3,
    8: 2,
    9: 1
}

results_dict = {}
place = 1
current_event = ""

for index, row in master_df.iterrows():
    school = row['School']
    event = row['event']
    
    if current_event != event:
        current_event = event
        place = 1
    else:
        place = place + 1
    
    #print(f"Row {index}: School={row['School']}, {row['event']}")

    if place < 10:
        if school in results_dict:
            results_dict[school] = results_dict[school] + place_values[place]
        else:
            results_dict[school] = place_values[place]

sorted_dict = dict(sorted(results_dict.items(), key=lambda item: item[1], reverse=True))


for key, value in sorted_dict.items():
    print(f"{key}: {value}")


Franklin Central: 39
Lawrence North: 37
Avon: 36
Warsaw Community: 36
Merrillville: 32
North Central (Indianapolis): 31
Indianapolis Bishop Chatard: 29
Bloomington North: 29
Chesterton: 21
Center Grove: 20
Mt. Vernon (Fortville): 18
Westview: 18
Brownsburg: 18
Churubusco: 17
Columbus North: 16
Concord: 15
Greenwood Community: 14
Fishers: 13
Noblesville: 12
Evansville North: 10
Homestead: 10
Northridge: 10
Mishawaka: 10
Crown Point: 10
Lawrence Central: 9
Brebeuf Jesuit Preparatory: 9
Bloomington South: 8
Warren Central: 8
Floyd Central: 8
Indianapolis Cathedral: 7
Tri-West Hendricks: 7
Elkhart: 7
Ben Davis: 7
Andrean: 7
Batesville: 7
Heritage Christian: 7
Carmel: 6
Jennings County: 6
Springs Valley: 6
New Haven: 6
Owen Valley: 6
Southport: 6
Henryville: 6
Westfield: 6
LaPorte: 6
NorthWood: 5
Charlestown: 5
Fort Wayne Snider: 5
Lakewood Park Christian: 5
Plainfield: 5
West Noble: 5
Salem: 5
Lebanon: 5
Wabash: 4
Evansville Mater Dei: 4
Fort Wayne Bishop Luers: 4
Cascade: 4
Jeffersonville